# Agentic RAG with Llamaindex and LLM

#### 1. pip install necessary libraries

In [ ]:
!pip install fastembed llama-index-vector-stores-chroma llama-index-llms-mistralai llama-index-embeddings-fastembed llama-index-readers-file
!pip install llama_index.vector_stores.chroma llama_index.vector_stores.chroma llama_index.llms.mistralai llama_index.embeddings.fastembed

#### 2. Import necessary Libraries

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from llama_index.core.agent import AgentRunner
from llama_index.core.objects import ObjectIndex
from llama_index.core.tools import FunctionTool
from llama_index.llms.mistralai import MistralAI
from typing import List,Optional
import os
#from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#### 3. Download Dataset

urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]

In [ ]:
def download_file(url, file_path):
    """Downloads a file from a given URL and saves it to the specified file path.

    Args:
        url: The URL of the file to download.
        file_path: The path to save the downloaded file.
    """

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    with open(file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)

    print(f"Downloaded file from {url} to {file_path}")


for url, paper in zip(urls, papers):
    download_file(url, paper)

#### 4 . Store vectos in chroma db

The object index supports integrations with any existing storage backend in LlamaIndex.

The following section walks through how to set that up using Chroma db as an example.

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### 5 . Define the model with API key

In [ ]:
os.environ["MISTRAL_API_KEY"] = ""
Settings.llm = MistralAI(model="mistral-large-latest")
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

#### 6 . Function to Create Summary tool and vector Tool

For each document, we set up two types of indices:

- Vector Index: This is used for semantic search, which means it helps in understanding and retrieving information based on the meaning and context of the query.
- Summary Index: This is used for generating summaries of the document, providing a concise overview of the content.

In [ ]:
# TODO: abstract all of this into a function that takes in a PDF file name
def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context
    )
    summary_index = SummaryIndex(nodes)

    def vector_query(
        query: str, page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts, condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}", fn=vector_query
    )

    def summary_query(
        query: str,
    ) -> str:
        """Perform a summary of document
        query (str): the string query to be embedded.
        """
        summary_engine = summary_index.as_query_engine(
            response_mode="tree_summarize",
            use_async=True,
        )

        response = summary_engine.query(query)
        return response

    summary_tool = FunctionTool.from_defaults(
        fn=summary_query, name=f"summary_tool_{name}"
    )

    return vector_query_tool, summary_tool

#### 7 . Index the document

In [ ]:
paper_to_tools_dict = {}
papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

The ObjectIndex class is designed to enable the indexing of various Python objects, making it highly versatile for numerous applications.

To create an ObjectIndex, you need two components: the index itself and a ObjectNodeMapping. The ObjectNodeMapping serves as a bridge, allowing you to map between a node and its corresponding object, and vice versa. Alternatively, the ObjectIndex class provides a from_objects() class method that simplifies the creation process by allowing you to construct an ObjectIndex directly from a collection of objects.

In [ ]:
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

Additionally, you can enhance the functionality of an ObjectIndex retriever by adding node-postprocessors. These postprocessors, such as rerankers, provide convenient ways to further process or refine the retrieval results

In [ ]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)
tools = obj_retriever.retrieve("compare and contrast the papers self rag and metagpt")
#
print(tools[0].metadata)
print(tools[1].metadata)

#### 8. Use our Mistral agent, powered by function calling capabilities.

In [ ]:
# Create Agent Runner
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], llm=Settings.llm, verbose=True
)
agent = AgentRunner(agent_worker)

#### 9 . Test agent

In [ ]:
response = agent.query(
    "what are agent roles in MetaGPT, "
    "and then how they communicate with each other."
)